In [2]:
import numpy as np
from evidently.metrics import EmbeddingsDriftMetric, Comment
from evidently.report import Report
from evidently.metrics.data_drift.embedding_drift_methods import model, ratio, mmd
import pickle
import pandas as pd
from evidently import ColumnMapping
from tensorflow.keras.models import load_model

In [3]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [4]:
from deepchecks.vision import classification_dataset_from_directory
import json
import matplotlib.pyplot as plt
from deepchecks.vision.suites import full_suite
from deepchecks.vision.suites import full_suite

/Users/ximenamoure/tensorflow-test/env/lib/python3.8/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning:

`TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.



In [5]:
client = MlflowClient()

In [6]:
""" # Load embeddings from .npy files
embeddings_train = np.load("/Users/ximenamoure/Desktop/drift_last/data/features_livechess_model_ref.npy")
embeddings_test = np.load("/Users/ximenamoure/Desktop/drift_last/data/features_livechess_model_split0.npy") """

' # Load embeddings from .npy files\nembeddings_train = np.load("/Users/ximenamoure/Desktop/drift_last/data/features_livechess_model_ref.npy")\nembeddings_test = np.load("/Users/ximenamoure/Desktop/drift_last/data/features_livechess_model_split0.npy") '

In [7]:
reference_split_name = "SPLIT5"
current_split_name = "SPLIT5_trans"
file_name = 'embeddings_drift_s5_vs_s5trans_auto_v3.html'
full_path = '/Users/ximenamoure/Desktop/drift_last/drift_reports/autoencoder_v3/'

In [8]:
# Load embeddings from .pkl files
with open("/Users/ximenamoure/Desktop/drift_last/embeddings_autoencoder_v3/x_ref_embeddings_autov3.pkl", "rb") as f:
    embeddings_train = pickle.load(f)


In [9]:
embeddings_train

array([[ -1.7055386 ,  -3.1910574 ,   5.840725  , ...,   7.177822  ,
         -2.860446  ,  -8.658086  ],
       [  3.1875327 ,   7.2673125 ,  10.117247  , ...,  20.731285  ,
          0.6189734 ,  -8.819734  ],
       [  6.7730303 ,   0.2008654 ,  -3.237001  , ...,  -5.3264484 ,
         -8.221491  ,  10.008629  ],
       ...,
       [ -1.3509235 ,  -0.03419232,   3.2369564 , ...,  -4.6609635 ,
         -4.1752343 ,  13.211972  ],
       [  0.45131055,   2.4122355 ,   1.7833797 , ...,   1.1689043 ,
          4.714209  ,  -4.679858  ],
       [  1.6744896 ,   2.9815855 ,  -1.1983256 , ..., -11.513319  ,
         -9.4208355 ,  11.057816  ]], dtype=float32)

In [10]:
with open("/Users/ximenamoure/Desktop/drift_last/embeddings_autoencoder_v3/ref_embeddings_autov3_5_MORE_MORE_MORE_MORE_trans.pkl", "rb") as f:
    embeddings_test = pickle.load(f)

In [11]:
if not isinstance(embeddings_train, np.ndarray):
    embeddings_train = np.array(embeddings_train)

In [12]:
if not isinstance(embeddings_test, np.ndarray):
    embeddings_test = np.array(embeddings_test)

In [13]:
# Convert embeddings to pandas DataFrames
embeddings_train_df = pd.DataFrame(embeddings_train, columns=[str(i) for i in range(embeddings_train.shape[1])])


In [14]:
embeddings_train_df

,0,1,2,3,4,5,6,7,8,9,...,340,341,342,343,344,345,346,347,348,349
0,-1.705539,-3.191057,5.840725,-3.539641,8.410621,4.486495,-7.927293,-4.156708,-2.301074,5.700826,...,-1.937374,1.842402,9.085405,-6.165084,-1.610895,-5.492609,7.589061,7.177822,-2.860446,-8.658086
1,3.187533,7.267313,10.117247,-13.261747,11.177268,5.100955,-0.209387,8.916093,5.039375,0.196834,...,-9.484519,-1.656791,0.995803,-1.220824,-5.502778,-1.123627,11.688406,20.731285,0.618973,-8.819734
2,6.773030,0.200865,-3.237001,3.135001,-0.763642,4.473423,3.034288,-8.284781,6.331875,2.160258,...,5.169977,5.223843,-3.369831,7.485655,-1.291078,-4.131930,-10.325645,-5.326448,-8.221491,10.008629
3,0.006819,9.488745,9.128934,6.648500,3.733341,-5.612949,-6.932809,5.187028,-3.745378,5.867865,...,-3.852876,-3.998436,5.732134,-8.258405,-4.545065,-8.605547,8.902964,1.965866,-1.538123,-14.637053
4,-3.891679,4.731290,-10.813214,-9.796288,-2.802206,11.138666,9.794683,-3.995347,5.909670,-11.204248,...,4.896925,-7.211381,-12.280624,-0.960214,0.063644,-6.083097,-3.174436,-2.783792,-6.422989,10.152508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48312,2.065873,-0.351717,-0.670688,-1.264324,6.282114,3.020775,-4.316151,0.689689,0.350635,4.815616,...,-6.349173,1.067961,7.384382,-1.300428,1.285263,-6.442620,4.845827,4.814539,0.760935,-12.983901
48313,-1.908342,0.606396,-1.662538,1.757597,1.895130,-2.086223,7.506543,0.901117,2.771191,1.639333,...,6.212283,3.141811,-6.736985,8.919179,-1.340759,3.067233,-18.975136,-1.728128,-5.729904,8.363076
48314,-1.350924,-0.034192,3.236956,-3.921557,5.121882,5.026652,8.696358,-8.964975,-1.708732,-1.734353,...,3.188804,-3.930628,-5.477764,14.590614,1.278780,-3.584880,-9.712955,-4.660964,-4.175234,13.211972
48315,0.451311,2.412235,1.783380,0.554370,2.964405,0.103030,-4.109717,-2.265414,-2.933920,9.955810,...,-2.078513,-0.593230,4.188463,-3.260260,4.449543,-8.210453,5.521957,1.168904,4.714209,-4.679858


In [15]:
embeddings_test_df = pd.DataFrame(embeddings_test, columns=[str(i) for i in range(embeddings_test.shape[1])])

In [16]:
embeddings_test_df

,0,1,2,3,4,5,6,7,8,9,...,340,341,342,343,344,345,346,347,348,349
0,-1.705539,-3.191057,5.840725,-3.539641,8.410621,4.486495,-7.927293,-4.156708,-2.301074,5.700826,...,-1.937374,1.842402,9.085405,-6.165084,-1.610895,-5.492609,7.589061,7.177822,-2.860446,-8.658086
1,3.187533,7.267313,10.117247,-13.261747,11.177268,5.100955,-0.209387,8.916093,5.039375,0.196834,...,-9.484519,-1.656791,0.995803,-1.220824,-5.502778,-1.123627,11.688406,20.731285,0.618973,-8.819734
2,6.773030,0.200865,-3.237001,3.135001,-0.763642,4.473423,3.034288,-8.284781,6.331875,2.160258,...,5.169977,5.223843,-3.369831,7.485655,-1.291078,-4.131930,-10.325645,-5.326448,-8.221491,10.008629
3,0.006819,9.488745,9.128934,6.648500,3.733341,-5.612949,-6.932809,5.187028,-3.745378,5.867865,...,-3.852876,-3.998436,5.732134,-8.258405,-4.545065,-8.605547,8.902964,1.965866,-1.538123,-14.637053
4,-3.891679,4.731290,-10.813214,-9.796288,-2.802206,11.138666,9.794683,-3.995347,5.909670,-11.204248,...,4.896925,-7.211381,-12.280624,-0.960214,0.063644,-6.083097,-3.174436,-2.783792,-6.422989,10.152508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48312,2.065873,-0.351717,-0.670688,-1.264324,6.282114,3.020775,-4.316151,0.689689,0.350635,4.815616,...,-6.349173,1.067961,7.384382,-1.300428,1.285263,-6.442620,4.845827,4.814539,0.760935,-12.983901
48313,-1.908342,0.606396,-1.662538,1.757597,1.895130,-2.086223,7.506543,0.901117,2.771191,1.639333,...,6.212283,3.141811,-6.736985,8.919179,-1.340759,3.067233,-18.975136,-1.728128,-5.729904,8.363076
48314,-1.350924,-0.034192,3.236956,-3.921557,5.121882,5.026652,8.696358,-8.964975,-1.708732,-1.734353,...,3.188804,-3.930628,-5.477764,14.590614,1.278780,-3.584880,-9.712955,-4.660964,-4.175234,13.211972
48315,0.451311,2.412235,1.783380,0.554370,2.964405,0.103030,-4.109717,-2.265414,-2.933920,9.955810,...,-2.078513,-0.593230,4.188463,-3.260260,4.449543,-8.210453,5.521957,1.168904,4.714209,-4.679858


In [17]:
column_mapping = ColumnMapping(
    embeddings={
        'embeddings_subset': embeddings_train_df.columns.tolist()
    }
)

In [18]:
column_mapping

ColumnMapping(target='target', prediction='prediction', datetime='datetime', id=None, numerical_features=None, categorical_features=None, datetime_features=None, target_names=None, task=None, pos_label=1, text_features=None, embeddings={'embeddings_subset': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121

In [19]:
embeddings_drift_metric_mmd = EmbeddingsDriftMetric(
    'embeddings_subset',
    drift_method = mmd(
                            threshold = 0.015,
                            bootstrap = None,
                            quantile_probability = 0.95,
                            pca_components = None,
                        )
)

embeddings_drift_metric_ratio = EmbeddingsDriftMetric(
    'embeddings_subset',
    drift_method = ratio(
                            component_stattest = 'wasserstein',
                            component_stattest_threshold = 0.1,
                            threshold = 0.2,
                            pca_components = None,
                        )
) 

embeddings_drift_metric_model = EmbeddingsDriftMetric(
    'embeddings_subset',
    drift_method = model(
                            threshold = 0.55,
                            bootstrap = None,
                            quantile_probability = 0.95,
                            pca_components = None,
                        )
) 

In [20]:
model_description = f"""
#  Drift detection between {reference_split_name} and {current_split_name}
"""


In [21]:
# Create the report and add the metric
report = Report(metrics=[
    Comment(model_description),
    embeddings_drift_metric_mmd,
    embeddings_drift_metric_ratio,
    embeddings_drift_metric_model,
    #embeddings_drift_metric_distance
])

In [22]:
# Run the report to calculate the drift
report.run(reference_data=embeddings_train_df, current_data=embeddings_test_df, column_mapping=column_mapping)

In [23]:
# Save the report as HTML for a detailed analysis
path = f"{full_path}{file_name}"
report.save_html(path)

In [24]:
# Extract the results as a dictionary
report_json = report.as_dict()

In [25]:
report_json

{'metrics': [{'metric': 'Comment', 'result': {}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_subset',
    'drift_score': 0.000824087574375354,
    'drift_detected': False,
    'method_name': 'mmd'}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_subset',
    'drift_score': 0.28285714285714286,
    'drift_detected': True,
    'method_name': 'ratio'}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_subset',
    'drift_score': 0.5202347700958527,
    'drift_detected': False,
    'method_name': 'model'}}]}

In [26]:
# Check if drift was detected
# Assuming you want to check the result for the MMD method first
mmd_result = report_json['metrics'][1]['result']['drift_detected']
ratio_result = report_json['metrics'][2]['result']['drift_detected']
model_result = report_json['metrics'][3]['result']['drift_detected']
#distance_result = report_json['metrics'][4]['result']['drift_detected']

